In [3]:
from __future__ import print_function

#torch
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#torchvision
import torchvision.models as models

#image
from PIL import Image
import cv2

#jupyter
from ipywidgets import FloatProgress
from IPython.display import display


#os
import os
import os.path as path
import glob

#math
import math
import numpy as np
import random

In [4]:
class DummyMultiFrame(nn.Module):
    def __init__(self, num_classes=6, num_frame=1):
        super(DummyMultiFrame, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3*num_frame, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    
    def copyParameters(self, alexnetModel):
        for i, a in enumerate(alexnetModel.features):
            if type(a) is torch.nn.modules.conv.Conv2d:
                self.features[i].weight = a.weight
                self.features[i].bias   = a.bias

In [30]:
def conv3x3(in_planes, out_planes, stride=1, groups=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups , bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, groups, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride, groups=groups)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, groups=groups)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

    

class MultiFrameResNet(nn.Module):

    def __init__(self, block, layers, sequence, groups, num_classes=6):
        self.inplanes = 22*sequence
        super(MultiFrameResNet, self).__init__()
        self.conv1 = nn.Conv2d(3*sequence, 22*sequence, kernel_size=3, stride=1, padding=1,
                               bias=False,groups=groups)
        self.bn1 = nn.BatchNorm2d(22*sequence)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 22*sequence, layers[0], groups)
        self.layer2 = self._make_layer(block, 44*sequence, layers[1], 1, stride=2)
        self.layer3 = self._make_layer(block, 264, layers[2], 1, stride=2)
        self.layer4 = self._make_layer(block, 528, layers[3], 1, stride=2)
        #self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(528 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks,  groups, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False, groups=groups),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, groups, stride=stride, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        #x = self.avgpool(x)
        x = nn.AvgPool2d(x.size(2))(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [28]:
class MultiFrameCNN(nn.Module):
    def __init__(self, nbFrame, nbChannel=3, nbClasse=6, groups=-1):
        super(MultiFrameCNN, self).__init__()
        
        if groups <= 0:
            self.groups = nbFrame
        else:
            self.groups = groups
        self.nbChannel = nbChannel
        self.nF = nbFrame
        self.features = nn.Sequential(
            nn.Conv2d(self.nbChannel*self.nF, 64*self.nF, kernel_size=5, stride=2, padding=1, groups=self.groups),
            nn.BatchNorm2d(64*self.nF),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64*self.nF, 192*self.nF, kernel_size=3, groups=self.groups),
            nn.BatchNorm2d(192*self.nF),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192*self.nF, 256*self.nF, kernel_size=1, padding=1),
            nn.BatchNorm2d(256*self.nF),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256*self.nF, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        #self.classifier = nn.Linear(32,nbClasse)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(192*3*3, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048,6),
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal(m.weight.data)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        #return x
        x = x.view(x.size(0), 192*3*3)
        return self.classifier(x)

In [29]:
if __name__=="__main__":
    SequenceLength = 3
    #m = MultiFrameResNet(BasicBlock, [2,2,2,2], 3)
    m = MultiFrameCNN(3, groups=3)
    x = torch.Tensor(1,SequenceLength*3,128,128)
    print(m(Variable(x)).size())

torch.Size([1, 6])
